# Data Processing

## 오픈 도메인 대화 테스크를 위한 데이터 처리
### dataset : https://github.com/songys/Chatbot_data

In [8]:
import sentencepiece as spm
import pandas as pd
import numpy as np

train_data = pd.read_csv('./data/dataset/ChatbotData.csv')
train_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


## sentence piece 로 vocab생성
## SentencePiece: A simple and language independent subword tokenizer and detokenizer for Neural Text Processing
### * Taku Kudo, John Richardson, Google

RNN은 기본적으로 vocab의 크기가 계산량에 영향을 주고 있습니다.
그래서 적당한 크기의 vocab을 사용하게 됩니다. 문제는 여기서 많이 발생합니다.
우리는 vocab을 만들때 미등록 단어가 발생하게 되고 실제로 입력으로 들어왔을때 UNK토큰으로 대체하게 됩니다.
이 과정에서 정보의 손실이 발생하고 성능의 문제를 일으킬수 있습니다.
그런 점을 보완하고자 sentencepiece를 tokenizer로 사용하려고 합니다.
sentencepiece의 기본 아이디어는 단어(word)의 부분단어(subword)로 모든 단어를 표현하고자 하는게 아이디어입니다.
이때 사용하는게 단어들의 빈도수를 사용하여 subword로 나눌지 말지를 판단하게 됩니다.

In [9]:
corpus = "data/dataset/chit-chat_corpus.txt"
prefix = "chatbot"
vocab_size = 16000
spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" + 
    " --model_type=bpe" +
    " --max_sentence_length=999999" + # 문장 최대 길이
    " --pad_id=0 --pad_piece=[PAD]" + # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" + # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" + # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" + # end of sequence (3)
    " --user_defined_symbols=[SEP],[CLS],[MASK]") # 사용자 정의 토큰

# Load & Test

In [10]:
vocab_file = "chatbot.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)
line = "3박4일 정도 놀러가고 싶다"
pieces = vocab.encode_as_pieces(line)
ids = vocab.encode_as_ids(line)


print(line)
print(pieces)
print(ids)

3박4일 정도 놀러가고 싶다
['▁3', '박', '4', '일', '▁정도', '▁놀러가고', '▁싶다']
[473, 15432, 15399, 14972, 982, 3503, 201]


In [11]:
import os
import sys
import json
import torch
import random
import torch.utils.data as data
import numpy as np
import pandas as pd

from torch.autograd import Variable 
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

from tqdm import tqdm
from tqdm import trange
import torch.nn.functional as F
#from torch.utils.tensorboard import SummaryWriter

from src.model import save

In [12]:
class Preprocessing:
    '''
    데이터의 최대 token길이가 10이지만
    실제 환경에서는 얼마의 길이가 들어올지 몰라 적당한 길이 부여
    '''
    
    def __init__(self, max_len = 20):
        self.max_len = max_len
        self.PAD = 0
    
    def pad_idx_sequencing(self, q_vec):
        q_len = len(q_vec)
        diff_len = q_len - self.max_len
        if(diff_len>0):
            q_vec = q_vec[:self.max_len]
            q_len = self.max_len
        else:
            pad_vac = [0] * abs(diff_len)
            q_vec += pad_vac

        return q_vec
    
    def make_batch(self):
        pass

class ChitChatDataset(data.Dataset):
    def __init__(self, x_tensor, y_tensor, labels):
        super(ChitChatDataset, self).__init__()

        self.x = x_tensor
        self.y = y_tensor
        self.labels = labels
        
    def __getitem__(self, index):
        return self.x[index], self.y[index], self.labels[index]

    def __len__(self):
        return len(self.x)
    
class MakeDataset:
    def __init__(self):
        
        self.chitchat_data_dir = "./data/dataset/ChatbotData.csv"
        
        self.prep = Preprocessing()
        vocab_file = "chatbot.model"
        self.transformers_tokenizer = spm.SentencePieceProcessor()
        self.transformers_tokenizer.load(vocab_file)
    
    def encode_dataset(self, dataset):
        token_dataset = []
        for data in dataset:
            token_dataset.append( [2] + self.transformers_tokenizer.encode_as_ids(data) + [3])
        return token_dataset

    def make_chitchat_dataset(self, train_ratio = 0.8):
        chitchat_dataset = pd.read_csv(self.chitchat_data_dir)
        Qs = chitchat_dataset["Q"].tolist()
        As = chitchat_dataset["A"].tolist()
        label = chitchat_dataset["label"].tolist()
        
        Qs = self.encode_dataset(Qs)
        As = self.encode_dataset(As)
        
        self.prep.max_len = 40
        x, y = [], []
        for q, a in zip(Qs,As):
            x.append(self.prep.pad_idx_sequencing(q))
            y.append(self.prep.pad_idx_sequencing(a))
        x = torch.tensor(x)
        y = torch.tensor(y)
        x_len = x.size()[0]
        train_size = int(x_len*train_ratio)
        
        if(train_ratio == 1.0):
            train_x = x[:train_size]
            train_y = y[:train_size]
            train_label = label[:train_size]
            train_dataset = ChitChatDataset(train_x,train_y,train_label)
            return train_dataset, None
        else:
            train_x = x[:train_size]
            train_y = y[:train_size]
            train_label = label[:train_size]

            test_x = x[train_size+1:]
            test_y = y[train_size+1:]
            test_label = label[train_size+1:]

            train_dataset = ChitChatDataset(train_x,train_y,train_label)
            test_dataset = ChitChatDataset(test_x,test_y,test_label)

            return train_dataset, test_dataset

In [14]:
dataset = MakeDataset()

train_dataset, test_dataset = dataset.make_chitchat_dataset(1.0)

train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
#test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=True)

# Attention Is All You Need
## * Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
### tensorflow transformer chatbot code : https://blog.tensorflow.org/2019/05/transformer-chatbot-tutorial-with-tensorflow-2.html

In [15]:
from torch.nn import Transformer
from torch import nn
import torch
import math
from tqdm import tqdm
class Tformer(nn.Module):
    def __init__(self, num_tokens, dim_model, num_heads, dff, num_layers, dropout_p=0.5):
        super(Tformer, self).__init__()
        self.transformer = Transformer(dim_model, num_heads, dim_feedforward=dff, num_encoder_layers=num_layers, num_decoder_layers=num_layers,dropout=dropout_p)
        self.pos_encoder = PositionalEncoding(dim_model, dropout_p)
        self.encoder = nn.Embedding(num_tokens, dim_model)

        self.pos_encoder_d = PositionalEncoding(dim_model, dropout_p)
        self.encoder_d = nn.Embedding(num_tokens, dim_model)

        self.dim_model = dim_model
        self.num_tokens = num_tokens

        self.linear = nn.Linear(dim_model, num_tokens)

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def forward(self, src, tgt, srcmask, tgtmask, srcpadmask, tgtpadmask):
        src = self.encoder(src) * math.sqrt(self.dim_model)
        src = self.pos_encoder(src)

        tgt = self.encoder_d(tgt) * math.sqrt(self.dim_model)
        tgt = self.pos_encoder_d(tgt)

        output = self.transformer(src.transpose(0,1), tgt.transpose(0,1), srcmask, tgtmask, src_key_padding_mask=srcpadmask, tgt_key_padding_mask=tgtpadmask)
        output = self.linear(output)
        return output

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

def gen_attention_mask(x):
    mask = torch.eq(x, 0)
    return mask

In [76]:
model = Tformer(
     num_tokens=vocab_size+7, dim_model=256, num_heads=8, dff=512, num_layers=2, dropout_p=0.1
 ).cuda()

In [77]:
lr = 1e-4
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
MAX_LENGTH = 40

In [78]:
epoch = 70
save_dir = "./data/pretraining/4_chitchat_transformer_model/"
save_prefix = "chitchat_transformer"
prev_loss_all = float("inf")
train_steps = 0
test_steps = 0
model.train()
for i in range(epoch):
    batchloss = 0.0
    progress = tqdm(train_dataloader)
    for (inputs, y, _) in progress:
        optimizer.zero_grad()

        dec_inputs = y[:,:-1]
        outputs = y[:,1:]
        
        src_mask = model.generate_square_subsequent_mask(MAX_LENGTH).cuda()
        src_padding_mask = gen_attention_mask(inputs).cuda()
        tgt_mask = model.generate_square_subsequent_mask(MAX_LENGTH-1).cuda()
        tgt_padding_mask = gen_attention_mask(dec_inputs).cuda()

        result = model(inputs.long().cuda(), dec_inputs.long().cuda(), src_mask, tgt_mask, src_padding_mask,tgt_padding_mask)
        loss = criterion(result.permute(1,2,0), outputs.long().cuda())
        progress.set_description("{:0.3f}".format(loss))

        train_steps += 1
        loss.backward()
        optimizer.step()
        batchloss += loss
    
    print("train epoch:",i+1,"|","loss:",batchloss.cpu().item() / len(train_dataloader))

#     model.eval()
#     test_batchloss = 0.0
#     progress_test = tqdm(test_dataloader)
#     for (inputs, y, _) in progress_test:

#         dec_inputs = y[:,:-1]
#         outputs = y[:,1:]
        
#         src_mask = model.generate_square_subsequent_mask(MAX_LENGTH).cuda()
#         src_padding_mask = gen_attention_mask(inputs).cuda()
#         tgt_mask = model.generate_square_subsequent_mask(MAX_LENGTH-1).cuda()
#         tgt_padding_mask = gen_attention_mask(dec_inputs).cuda()

#         result = model(inputs.long().cuda(), dec_inputs.long().cuda(), src_mask, tgt_mask, src_padding_mask,tgt_padding_mask)
 
#         loss = criterion(result.permute(1,2,0), outputs.long().cuda())
#         progress_test.set_description("{:0.3f}".format(loss.cpu().item()))

#         test_steps += 1
#         test_batchloss += loss.cpu().item()
#     loss_all = test_batchloss/len(test_dataloader)
#     print("test epoch:",i+1,"|","loss:",loss_all)
#     model.train()
#     if(loss_all<prev_loss_all):
#         prev_loss_all = loss_all
#         save(model, save_dir, save_prefix + "_" + str(round(loss_all,6)), i)

1.356:   1%|          | 1/93 [00:00<00:11,  8.33it/s]

train epoch: 1 | loss: 2.3234646704889115


1.025:   1%|          | 1/93 [00:00<00:11,  8.20it/s]

train epoch: 2 | loss: 1.1157909106182795


0.916:   1%|          | 1/93 [00:00<00:11,  8.26it/s]

...

train epoch: 68 | loss: 0.28686312193511637


0.259:   1%|          | 1/93 [00:00<00:11,  7.75it/s]

train epoch: 69 | loss: 0.2803343906197497


0.278: 100%|██████████| 93/93 [00:11<00:00,  8.12it/s]

train epoch: 70 | loss: 0.2735347952893985


In [79]:
loss

tensor(0.2783, device='cuda:0', grad_fn=<NllLoss2DBackward>)

In [80]:
save(model, save_dir, save_prefix + "_" + str(round(loss.cpu().item(),6)), i)

In [81]:
def preprocess_sentence(sentence):
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    return sentence

def evaluate(sentence):
    sentence = preprocess_sentence(sentence)
    input = torch.tensor([[2] + vocab.encode_as_ids(sentence) + [3]]).cuda()
    output = torch.tensor([[2]]).cuda()

    # 디코더의 예측 시작
    model.eval()
    for i in range(MAX_LENGTH):
        src_mask = model.generate_square_subsequent_mask(input.shape[1]).cuda()
        tgt_mask = model.generate_square_subsequent_mask(output.shape[1]).cuda()

        src_padding_mask = gen_attention_mask(input).cuda()
        tgt_padding_mask = gen_attention_mask(output).cuda()

        predictions = model(input, output, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask).transpose(0,1)
        # 현재(마지막) 시점의 예측 단어를 받아온다.
        predictions = predictions[:, -1:, :]
        predicted_id = torch.LongTensor(torch.argmax(predictions.cpu(), axis=-1))


        # 만약 마지막 시점의 예측 단어가 종료 토큰이라면 예측을 중단
        if torch.equal(predicted_id[0][0], torch.tensor(3)):
            break

        # 마지막 시점의 예측 단어를 출력에 연결한다.
        # 이는 for문을 통해서 디코더의 입력으로 사용될 예정이다.
        output = torch.cat([output, predicted_id.cuda()], axis=1)

    return torch.squeeze(output, axis=0).cpu().numpy()

def predict(sentence):
    prediction = evaluate(sentence)
    predicted_sentence = vocab.Decode(list(map(int,[i for i in prediction if i < vocab_size+7])))

    print('Input: {}'.format(sentence))
    print('Output: {}'.format(predicted_sentence))

    return predicted_sentence

In [58]:
model.load_state_dict(torch.load("./data/pretraining/save/4_chitchat_transformer_model/chitchat_transformer_1.215381_steps_81.pt"))

model.eval()

Tformer(
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): _LinearWithBias(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=512, out_features=256, bias=True)
          (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
        (1): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): _LinearWithBias(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_features=512, bias=True)
       

In [82]:
result = predict("난 뭘 해야 할까?")

Input: 난 뭘 해야 할까?
Output: 정말 힘드신가봐요. 본인의 의사를 확실히 밝혀보세요 대한 두려움을 가지고 손해배상을 청구하세요.


In [83]:
result = predict("힘들다")

Input: 힘들다
Output: 이제 회사와 자신에 대해서 더 공부해서 자신감을 가지세요.


In [84]:
result = predict("난 혼자인게 좋아")

Input: 난 혼자인게 좋아
Output: 스트레스 받으시는 말고 적극적으로 장점을 찾아서 인정하고 호의를 보여보세요.


In [85]:
result = predict("결혼해줘")

Input: 결혼해줘
Output: 더 힘들 겠지만 못해요하는 어제 더 면밀히 더 면밀히 더 면밀히 걸리겠지만 해낼 수도 있어요. 작은하는 어제 더 힘들 겠지만요하는 어제 더 힘들 겠지만요하는 어제 더 힘들 겠지만요군요
